In [1]:
from astropy.coordinates import SkyCoord
from astropy.coordinates.name_resolve import NameResolveError
from astropy.table import Table, Row, vstack, unique, hstack
from astropy.time import Time
from astropy import units as u

from astroquery.vizier import Vizier

from IPython.display import clear_output

import numpy

from pathlib import Path

import pickle

In [2]:
DATA_PATH = Path("//stem-linux-homes/OSL-Telescope/data/users/Pipeline/PIRATE/")
NUM_CALIBRATION_ROWS = 10
CALIBRATION_CATALOGUE = 'I/284/out'
CALIBRATION_CATALOGUE_FIELDS = {
    'B': 'B1mag',
    'R': 'R1mag',
    'I': 'Imag',
}

In [3]:
try: 
    with open('data/processed_dates.pickle', 'rb') as processed_dates_file:
        processed_dates = pickle.load(processed_dates_file)
except FileNotFoundError:
    processed_dates = []

In [4]:
try: 
    with open('data/coords.pickle', 'rb') as coords_file:
        coords = pickle.load(coords_file)
except FileNotFoundError:
    coords = {}

To do:
- Take account of flags (maybe don't do this at this step and leave it for later)
- Calibration

In [5]:
new_dates = [p for p in DATA_PATH.glob('202?_??_??') if p not in processed_dates]
total_dates = len(new_dates)
obs_tables = {}

SPLIT_SIZE = 10

for date_i, date in enumerate(new_dates):
    for catalogue in date.glob('Catalogues/*_anm83_*.cat'):
        obs_meta = catalogue.stem.split('_')
        obs_meta = {
            'name': obs_meta[5],
            'band': obs_meta[7][0],
            'exposure': obs_meta[7][1:],
            'timestamp': Time(
                dict(zip(
                    ['year', 'month', 'day', 'hour', 'minute', 'second'],
                    map(int, obs_meta[9:15])
                )),
                format='ymdhms',
            ).jd,
        }
        if obs_meta['name'] not in coords:
            try:
                coords[obs_meta['name']] = SkyCoord.from_name(obs_meta['name'], parse=True)
            except NameResolveError:
                continue
        if obs_meta['band'] == 'V':
            # To do: How to handle the lack of V mags in USNO?
            continue
        clear_output(wait=True)
        print(f'[{date_i}/{total_dates}] Processing {catalogue.stem}')
        
        table = Table.read(catalogue, format='ascii.sextractor')
        table.rename_column('ALPHA_J2000', 'RA')
        table.rename_column('DELTA_J2000', 'Dec')
        
        table['separation'] = SkyCoord.guess_from_table(table).separation(coords[obs_meta['name']])
        table.sort('separation') #To do: use idxmin here instead of sorting
        target_row = table[0]
        
        # Do USNO vizier query for contents of table, then take the closest few matching rows
        # to use for calibration
        
        table.rename_column('RA', '_RAJ2000')
        table.rename_column('Dec', '_DEJ2000')
        Path(f"data/calibration_tables/{obs_meta['name']}").mkdir(parents=True, exist_ok=True)
        try:
            calibration_table = Table.read(f"data/calibration_tables/{obs_meta['name']}/{catalogue.stem}.ecsv")
        except FileNotFoundError:
            table['flux_diff'] = numpy.abs(table['FLUX_AUTO'] - target_row['FLUX_AUTO'])
            table.sort('flux_diff')

            calibration_rows = []

            total_iterations = int(len(table) / SPLIT_SIZE) + 1
            for i in range(total_iterations):
                sources = table[table['FLAGS'] == 0][i * SPLIT_SIZE : (i+1) * SPLIT_SIZE]
                if len(sources) == 0:
                    continue
                sources = vstack([ row for row in sources if row['NUMBER'] != target_row['NUMBER'] ])
                catalogue_results = Vizier.query_region(sources, radius=1e-4*u.deg, catalog=CALIBRATION_CATALOGUE)
                if len(catalogue_results) == 0:
                    continue

                # Reject any sources with multiple matches
                catalogue_matches = unique(catalogue_results[0], '_q', keep='none')

                # To do: Reject any known variables

                for catalogue_row in catalogue_matches:
                    calibration_rows.append(hstack([
                        sources[int(catalogue_row['_q']) - 1],
                        catalogue_row[[CALIBRATION_CATALOGUE_FIELDS[obs_meta['band']]]],
                    ]))

                if len(calibration_rows) >= NUM_CALIBRATION_ROWS:
                    break
            if len(calibration_rows) < NUM_CALIBRATION_ROWS:
                continue
            calibration_table = vstack(calibration_rows[:NUM_CALIBRATION_ROWS])
            calibration_table.write(f"data/calibration_tables/{obs_meta['name']}/{catalogue.stem}.ecsv")
        
        calibrated_mags = calibration_table[CALIBRATION_CATALOGUE_FIELDS[obs_meta['band']]] - 2.5 * numpy.log10(target_row['FLUX_AUTO'] / calibration_table['FLUX_AUTO'])
        
        out_table = Table(target_row)
        # To do: Maybe this should be a weighted average using the flux error?
        out_table.add_columns(
            [
                numpy.mean(calibrated_mags),
                numpy.std(calibrated_mags),
            ],
            names=['calibrated_mag', 'calibrated_mag_err'],
        )
        
        for key, val in obs_meta.items():
            out_table[key] = val
        
        if obs_meta['name'] not in obs_tables:
            try:
                obs_tables[obs_meta['name']] = Table.read(f"data/{obs_meta['name']}.ecsv")
            except FileNotFoundError:
                obs_tables[obs_meta['name']] = out_table
                continue
        obs_tables[obs_meta['name']] = vstack([obs_tables[obs_meta['name']], out_table])
        
    processed_dates.append(date)

[655/659] Processing PIRATE_419624_OSL_00_anm83_HD38451_00_B15_00_2021_10_21_04_34_56_OR_CB_PS


In [6]:
for name, table in obs_tables.items():
    table.write(f"data/{name}.ecsv", overwrite=True)

In [7]:
with open('data/processed_dates.pickle', 'wb') as processed_dates_file:
    pickle.dump(processed_dates, processed_dates_file)

In [8]:
with open('data/coords.pickle', 'wb') as coords_file:
    pickle.dump(coords, coords_file)